# Viz For Social Good Challenge

Total data : *224*

Goal
----
- The goal is to analyze where to focus efforts for recruitment
- Understand what sort of non-profits to partner with in the future to improve user engagement.

## Import Data

In [1]:
import pandas as pd
import numpy as np

from multiprocessing.pool import ThreadPool
from tqdm import tqdm
import time 
from time import sleep
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

## Read in the data

In [2]:
filename = 'data/Viz for Social Good Annual Poll (Responses) Raw Selected.csv'

data_raw = pd.read_csv(filename)

data_raw.head()

,ID,What country do you live in?,What is your age?,What is the highest level of education you have completed?,What industry do you work in?,Which option most closely relates to your job function?,What program(s) do you use to create data visualizations?,How many years of experience do you have working with data visualization tools or projects?,How did you originally hear about VFSG?,How much time do you want to spend on one submission?,Time Spent on each task,s
0,91,Australia,26 - 35 years old,Master's degree (Graduate),Marketing,Analyst,"Excel, Tableau, R, Python or Matlab",1-3 years,Twitter,6-10 hours,MID,NaN
1,94,Australia,18 - 25 years old,Bachelor's degree (undergraduate),Education,Other,"Tableau, Physical materials , R, Python or Matlab",Less than 1 year,Other,More than 10 hours,MAX,NaN
2,119,Australia,36 - 50 years old,Master's degree (Graduate),Other,Data Scientist,"Alteryx, Excel, Tableau, R, Python or Matlab, SQL",4-5 years,Through a friend/Colleague,More than 10 hours,MAX,NaN
3,145,Australia,36 - 50 years old,Master's degree (Graduate),IT and Software,Analyst,Tableau,4-5 years,Twitter,2-5 hours,MIN,NaN
4,148,Australia,>50 years old,Master's degree (Graduate),IT and Software,Analyst,Power BI,4-5 years,LinkedIn,More than 10 hours,MAX,NaN


If more than half the data is missing, I decided to drop the column

In [3]:
columns = []
for (colname,colval) in data_raw.iteritems():
    if data_raw[colname].isna().sum()<224/2:
        columns.append(colname)
        
data_tot = data_raw[columns]
data_tot.head()

,ID,What country do you live in?,What is your age?,What is the highest level of education you have completed?,What industry do you work in?,Which option most closely relates to your job function?,What program(s) do you use to create data visualizations?,How many years of experience do you have working with data visualization tools or projects?,How did you originally hear about VFSG?,How much time do you want to spend on one submission?,Time Spent on each task
0,91,Australia,26 - 35 years old,Master's degree (Graduate),Marketing,Analyst,"Excel, Tableau, R, Python or Matlab",1-3 years,Twitter,6-10 hours,MID
1,94,Australia,18 - 25 years old,Bachelor's degree (undergraduate),Education,Other,"Tableau, Physical materials , R, Python or Matlab",Less than 1 year,Other,More than 10 hours,MAX
2,119,Australia,36 - 50 years old,Master's degree (Graduate),Other,Data Scientist,"Alteryx, Excel, Tableau, R, Python or Matlab, SQL",4-5 years,Through a friend/Colleague,More than 10 hours,MAX
3,145,Australia,36 - 50 years old,Master's degree (Graduate),IT and Software,Analyst,Tableau,4-5 years,Twitter,2-5 hours,MIN
4,148,Australia,>50 years old,Master's degree (Graduate),IT and Software,Analyst,Power BI,4-5 years,LinkedIn,More than 10 hours,MAX


In [4]:
data = data_tot.drop(columns=['How many years of experience do you have working with data visualization tools or projects?',
                            'What is the highest level of education you have completed?',
                            'How much time do you want to spend on one submission?'])

data.head()

,ID,What country do you live in?,What is your age?,What industry do you work in?,Which option most closely relates to your job function?,What program(s) do you use to create data visualizations?,How did you originally hear about VFSG?,Time Spent on each task
0,91,Australia,26 - 35 years old,Marketing,Analyst,"Excel, Tableau, R, Python or Matlab",Twitter,MID
1,94,Australia,18 - 25 years old,Education,Other,"Tableau, Physical materials , R, Python or Matlab",Other,MAX
2,119,Australia,36 - 50 years old,Other,Data Scientist,"Alteryx, Excel, Tableau, R, Python or Matlab, SQL",Through a friend/Colleague,MAX
3,145,Australia,36 - 50 years old,IT and Software,Analyst,Tableau,Twitter,MIN
4,148,Australia,>50 years old,IT and Software,Analyst,Power BI,LinkedIn,MAX


To measure the engaement, we'll need to take into consideration the different countries.   
For each country we look at the different *industries* which should correspond to the *jobs* which in turn should explain the tools used and time spent on visualizations.

## Measuring metrics for engagement.

In [5]:
data.columns

Index(['ID', 'What country do you live in?', 'What is your age?',
       'What industry do you work in?',
       'Which option most closely relates to your job function?',
       'What program(s) do you use to create data visualizations?',
       'How did you originally hear about VFSG?', 'Time Spent on each task'],
      dtype='object')

In [6]:
cols = ['ID', 
        'What country do you live in?',
        'What industry do you work in?',
        'Which option most closely relates to your job function?']
users_work = data[cols]

users_work.dropna(subset=['What country do you live in?'], 
                  how='all', 
                  inplace=True)
users_work['What industry do you work in?'] = users_work['What industry do you work in?'].fillna('other')
users_work['Which option most closely relates to your job function?'] = users_work['Which option most closely relates to your job function?'].fillna('Other')

users_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213 entries, 0 to 212
Data columns (total 4 columns):
 #   Column                                                   Non-Null Count  Dtype 
---  ------                                                   --------------  ----- 
 0   ID                                                       213 non-null    int64 
 1   What country do you live in?                             213 non-null    object
 2   What industry do you work in?                            213 non-null    object
 3   Which option most closely relates to your job function?  213 non-null    object
dtypes: int64(1), object(3)
memory usage: 8.3+ KB


In [7]:
users_work.to_csv('data/users-wrt-work.csv')

In [8]:
users_work.head(8)

,ID,What country do you live in?,What industry do you work in?,Which option most closely relates to your job function?
0,91,Australia,Marketing,Analyst
1,94,Australia,Education,Other
2,119,Australia,Other,Data Scientist
3,145,Australia,IT and Software,Analyst
4,148,Australia,IT and Software,Analyst
5,174,Australia,IT and Software,Analyst
6,215,Australia,Retail,Analyst
7,54,Belgium,Non-profit,Analyst


In [9]:
tot = len(users_work)
users_work['What country do you live in? %'] = (users_work.groupby(['What country do you live in?'])['What country do you live in?'].transform('count')/tot)*100
users_work['What industry do you work in? %'] =  (users_work.groupby(['What industry do you work in?'])['What industry do you work in?'].transform('count')/tot)*100
users_work['Which option most closely relates to your job function? %'] = (users_work.groupby(['Which option most closely relates to your job function?'])['Which option most closely relates to your job function?'].transform('count')/tot)*100

users_work = users_work.sort_values(by="What country do you live in? %",
                                   ascending=False)
          
# top_10 = users_work['What country do you live in? %'].unique()[:10]

users_work_summ = users_work[users_work['What country do you live in? %'] >= 8 ]
users_work_summ


users_work_summ.to_csv('data/users_work.csv')

In [10]:
len(users_work_summ[users_work_summ['What country do you live in?'] == 'United States']),len(users_work_summ[users_work_summ['What country do you live in?'] == 'Canada']),len(users_work_summ[users_work_summ['What country do you live in?'] == 'United Kingdom']),users_work_summ['What country do you live in?'].unique()

(84,
 21,
 18,
 array(['United States', 'Canada', 'United Kingdom'], dtype=object))

In [13]:
heard_res = data_tot.groupby(
            [
                'What country do you live in?',
                'How did you originally hear about VFSG?',
                # 'What industry do you work in?',
                # 'Which option most closely relates to your job function?',
                # 'What program(s) do you use to create data visualizations?'
            ]).agg(
                # reponses_count = ('What country do you live in?','count'),
                # count_industry = ('What industry do you work in?','count'),
                # count_job = ('Which option most closely relates to your job function?','count'),
                count_heard_of_VFSG = ('How did you originally hear about VFSG?','count')
                  ).reset_index()
            
heard_res.to_csv('data/users_work.csv')

In [ ]:
def count_words(sentence):
    # print(sentence, type(sentence))
    count = len(sentence.str.split(','))
    return count
count_words(data_tot['What program(s) do you use to create data visualizations?'])


tools = data_tot.groupby(
            [
                'What country do you live in?',
                'What program(s) do you use to create data visualizations?'
            ]
                         ).agg(
                             programs = (
                                 'What program(s) do you use to create data visualizations?',
                                 count_words
                                 )
                             )
                        
# tools.to_csv('data/tools.csv')

In [ ]:
c1 = data_tot[data_tot['What country do you live in?']=='Philippines']
# c1 = c1['What program(s) do you use to create data visualizations?'].to_list()
c1 = c1['Which option most closely relates to your job function?'].to_list()

    
# c1 = [item for sublist in c1 for item in sublist.split(',')]

c1

c1_df = pd.DataFrame(c1,
                     columns=['industry'])

print(c1_df.industry.to_list())
c1_df.groupby('industry').agg(count = ('industry','count'))

['Analyst', 'Educator/Teacher', 'Other', 'Project or Program Manager', 'Developer']


In [ ]:
# data_tot[['What program(s) do you use to create data visualizations?','What country do you live in?']].to_csv('data/tools.csv')